# Saturation - Maximum Find

In [ ]:
%load_ext autoreload
%autoreload 2
%reload_ext autoreload

In [ ]:
#from pymote.algorithms.saturation import Saturation
#from pymote.algorithms.maximumfind import MaxFind

## Algoritmi

### Shout

In [ ]:
from pymote.algorithm import NodeAlgorithm
from pymote.message import Message


class Flood(NodeAlgorithm):
    required_params = ('informationKey',)
    default_params = {'neighborsKey': 'Neighbors'}

    def initializer(self):
        ini_nodes = []
        for node in self.network.nodes():
            node.memory[self.neighborsKey] = \
                node.compositeSensor.read()['Neighbors']
            node.status = 'IDLE'
            if self.informationKey in node.memory:
                node.status = 'INITIATOR'
                ini_nodes.append(node)
        for ini_node in ini_nodes:
            self.network.outbox.insert(0, Message(header=NodeAlgorithm.INI,
                                                 destination=ini_node))

    def initiator(self, node, message):
        if message.header == NodeAlgorithm.INI:
            # default destination: send to every neighbor
            
            node.send(Message(header='Information',
                              data=node.memory[self.informationKey]))
            node.status = 'DONE'

    def idle(self, node, message):
        if message.header == 'Information':
            node.memory[self.informationKey] = message.data
            destination_nodes = list(node.memory[self.neighborsKey])
            # send to every neighbor-sender
            destination_nodes.remove(message.source)
            if destination_nodes:
                node.send(Message(destination=destination_nodes,
                                  header='Information',
                                  data=message.data))
        node.status = 'DONE'

    def done(self, node, message):
        pass

    STATUS = {
              'INITIATOR': initiator,
              'IDLE': idle,
              'ACTIVE': active,
              'DONE': done,
             }




### Saturation (Plug-in Full-Saturation)

In [ ]:
from pymote.algorithm import NodeAlgorithm
from pymote.message import Message

class Saturation(NodeAlgorithm): 
    
    #required_params = ('informationKey', 'treeKey',) # must have ','
    required_params = ('informationKey',) # must have ','
    default_params = {'neighborsKey': 'Neighbors','parentKey' : 'Parent'}

    def initializer(self):
        ini_nodes = []
        for node in self.network.nodes():
            node.memory[self.neighborsKey] = node.compositeSensor.read()['Neighbors']
            self.initialize(node)
            node.status = 'AVAILABLE'
            if self.informationKey in node.memory:
                node.status = 'AVAILABLE'
                ini_nodes.append(node)
        for ini_node in ini_nodes:
            self.network.outbox.insert(0, Message(header=NodeAlgorithm.INI,destination=ini_node))  # to je spontani impuls   

        
    def available(self, node, message):

        #inicijatori
        if message.header == NodeAlgorithm.INI: #Spontaneously
            node.send(Message(header='Activate', data='Activate'))
            #initialize() mislim da ipak treba biti u initalizeru
            if len(node.memory[self.neighborsKey])==1 : #ako je čvor list
                node.memory[self.parentKey] = list(node.memory[self.neighborsKey])
                updated_data=self.prepare_message(node)
                node.send(Message(header='M', data = updated_data, destination = node.memory[self.parentKey]))
                node.status = 'PROCESSING'
            else:
                node.status = 'ACTIVE' #izvrši se
                
        if message.header == 'Activate':
            destination_nodes = list(node.memory[self.neighborsKey])
            node.send(Message(header='Activate', data='Activate', destination=destination_nodes.remove(message.source)))
            #initialize() mislim da ipak treba biti u initalizeru            
            if len(node.memory[self.neighborsKey])==1 :
                node.memory[self.parentKey] = list(node.memory[self.neighborsKey])                
                updated_data=self.prepare_message(node)
                node.send(Message(header='M', data=updated_data, destination=node.memory[self.parentKey]))
                #dal prepare?
                node.status = 'PROCESSING'
            else:
                node.status='ACTIVE' #izvrši se
    
    def active(self, node, message):  
        

        if message.header=='M':
            self.process_message(node,message)
            print node.id
            print "prije:"
            print node.memory[self.neighborsKey]
            print len(node.memory[self.neighborsKey])
            ###ključno
            node.memory[self.neighborsKey].remove(message.source) # ne radi?
            print "poslije:"
            print node.memory[self.neighborsKey]
            print len(node.memory[self.neighborsKey])
            ##izgleda da neki ostanu bez susjeda kao posljedica ovoga, ne, trebali bi uci u processing jopš u availabele
            
            if len(node.memory[self.neighborsKey])==1 :
                print "jedan"
                node.memory[self.parentKey] = list(node.memory[self.neighborsKey])                
                updated_data=self.prepare_message(node)
                node.send(Message(header='M', data=updated_data, destination=node.memory[self.parentKey]))
                node.status = 'PROCESSING'

    def processing(self, node, message):
        if message.header=="M":           
            self.process_message(node,message)
            #self.resolve(node)
            node.staus='SATURATED'
        if message.header=="Notification":
            print "Nebi smio biti tu"
    def prepare_message(self,node):
        raise NotImplementedError
        
    def process_message(self, node, message):
        raise NotImplementedError

    def initialize(self, node):
        raise NotImplementedError
        
    def resolution():
        raise NotImplementedError         
    
    def resolve(self,node):
        node.status='SATURATED'
        #node.resolution()
        
    STATUS = {
              'AVAILABLE': available,
              'ACTIVE': active,
              'PROCESSING': processing,
              'SATURATED': resolution,
             }

### MaxFind

In [ ]:
# -*- coding: utf-8 -*-
from pymote.sensor import TempSensor

class MaxFind(Saturation):
    #required_params = ('dataKey',) 
    default_params = {'temperatureKey':'Temperature','maxKey':'Max'}

    def processing(self,node,message):
        if message.header=="M":
            self.process_message(node,message)
            node.status='SATURATED'
        
        if message.header=="Notification":
            print "HVALA TATA!"
            destination_nodes = node.memory[self.neighborsKey]
            
            print destination_nodes
            print node.memory[self.parentKey]
            
            self.process_message(node,message)
            destination_nodes.remove(node.memory[self.parentKey])            
            
            node.send(Message(header='Notification', data=node.memory[self.MaxKey]), destination=destination_nodes)
            
            if node.memory[self.TempKey]==message.data:
                node.status="MAXIMUM"
            else:
                node.status="SMALLER"
    
    def initialize(self, node):
        node.compositeSensor=(TempSensor,'Temperature')
        node.memory[self.temperatureKey]=node.compositeSensor.read()['Temperature']
        node.memory[self.maxKey]=node.memory[self.temperatureKey]   
        #return node.memory[self.maxKey] is not None    
    
    def prepare_message(self,node):
        return node.memory[self.maxKey]
                   
    def process_message(self,node,message):
        #print message.data
        if message.data>node.memory[self.maxKey]:
            node.memory[self.maxKey] = message.data
    
    def resolve(self,node):
        destination_nodes = list(node.memory[self.neighborsKey])
        #destination_nodes.remove(node.memory[self.parentKey]) garantira topologiju        
        node.send(Message(header='Notification', data=node.memory[self.maxKey], destination=destination_nodes))        
        print "TU SAM"
        if node.memory[self.temperatureKey] == node.memory[self.maxKey]:
            node.status='MAXIMUM'
        else :
            node.status='SMALL'
    
                                                 
    def mini():
        print "MAXIMUM"
    def larger():
        print "SMALLER"
    
    STATUS = {
              'MAXIMUM' : mini,
              'SMALLER' : larger,
              'AVAILABLE': Saturation.STATUS.get('AVAILABLE'),
              'ACTIVE': Saturation.STATUS.get('ACTIVE'),
              #'PROCESSING':Saturation.STATUS.get('PROCESSING'), #redefinirali smo processing
              'PROCESSING':processing, #redefinirali smo processing
              #'SATURATED':Saturation.STATUS.get('SATURATED'),
              'SATURATED':resolve,
             }    

### Random Mreža

In [ ]:
from pymote.networkgenerator import NetworkGenerator
from pymote.npickle import write_pickle, read_pickle
#net_gen = NetworkGenerator(3)
net_gen = NetworkGenerator(5)
#net_gen = NetworkGenerator(15)
#net_gen = NetworkGenerator(500)
net = net_gen.generate_random_network()

write_pickle(net,'mrezaRandom.tar.gz')

In [ ]:
net.show() #po gašenju izaziva Dead Kernel

### Mreža stablo

In [ ]:
from pymote.network import Network
net = Network()
root=net.add_node(pos=[300,500])
root.commRange=150
node=net.add_node(pos=[200,400])
node.commRange=150
node=net.add_node(pos=[400,400])
node.commRange=150

node=net.add_node(pos=[100,300])
node.commRange=150

node=net.add_node(pos=[250,300])
node.commRange=150
node=net.add_node(pos=[500,300])
node.commRange=150
node=net.add_node(pos=[450,200])
node.commRange=150
node=net.add_node(pos=[10,200])
node.commRange=150

from pymote.npickle import write_pickle, read_pickle
write_pickle(net,'mrezaStablo.tar.gz')

### Mreža Lanac (n = 4, m = 5)

In [ ]:
from pymote.network import Network
net = Network()
n = 5
u = 100

for i in range(0,n*u,u):
    node = net.add_node(pos=[1,i])
    node.commRange=110
    
from pymote.npickle import write_pickle, read_pickle
write_pickle(net,'mrezaList.tar.gz') 

### Mreža Zvijezda (n = 4, m = 5)

In [ ]:
from pymote.network import Network
net = Network()
n = 5
for i in range(100,(n+1)*100,100):
    node= net.add_node(pos=[i,i])
    node.commRange=200

node= net.add_node(pos=[100,300])
node.commRange=200
node= net.add_node(pos=[300,100])
node.commRange=200

from pymote.npickle import write_pickle, read_pickle
write_pickle(net,'mrezaZvijezda.tar.gz')

### Simulacija

In [1]:
%load_ext autoreload
%autoreload 2
%reload_ext autoreload

from pymote.algorithms.saturation import Saturation
from pymote.algorithms.maximumfind import MaxFind

In [2]:
from pymote.npickle import write_pickle, read_pickle
from pymote.simulation import Simulation

#net = read_pickle('mrezaRandom.tar.gz') 
#net = read_pickle('mrezaList.tar.gz')
#net = read_pickle('mrezaZvijezda.tar.gz')
net = read_pickle('mrezaStablo.tar.gz')

net.algorithms = ( (MaxFind,{'informationKey':'I'}), )

some_node = net.nodes()[4]# uzimamo prvi cvor u listi cvorova mreze
some_node.memory['I'] = 'Activate'

#some_node_2 = net.nodes()[1]# uzimamo prvi cvor u listi cvorova mreze
#some_node_2.memory['I'] = 'Activate'

sim = Simulation(net)
sim.run()

suma=0

print "Number of nodes:"
print len(list(net.nodes()))
print "Number of edges:"
print len(list(net.edges()))
print "Total number of messages:"
print suma

for node in net.nodes():
    #print node.memory['Max']
    print node.id
    print node.memory
    print node.status
    #suma=suma+node.memory['msgKey']

DEBUG   [simulation.py]: Simulation 0x10f04e18 created successfully.


DEBUG:pymote.simulation:Simulation 0x10f04e18 created successfully.


WARNING [__init__.py]: Environment variable PYMOTE_SETTINGS_MODULE is undefined, using global_settings.


DEBUG   [simulation.py]: [MaxFind] Step 1 finished


DEBUG:pymote.simulation:[MaxFind] Step 1 finished


...

DEBUG   [simulation.py]: [MaxFind] Algorithm finished


DEBUG:pymote.simulation:[MaxFind] Algorithm finished


INFO    [simulation.py]: Simulation has finished. There are no algorithms left to run. To run it from the start use sim.reset().


INFO:pymote.simulation:Simulation has finished. There are no algorithms left to run. To run it from the start use sim.reset().


Number of nodes:
8
Number of edges:
7
Total number of messages:
0
8
{'Neighbors': [<Node id=10>], 'Max': 30, 'Temperature': -15, 'Parent': [<Node id=10>]}
SATURATED
9
{'Neighbors': [<Node id=8>], 'Max': 30, 'Temperature': 30, 'Parent': [<Node id=8>]}
PROCESSING
10
{'Neighbors': [<Node id=8>], 'Max': 30, 'Temperature': 18, 'Parent': [<Node id=8>]}
SATURATED
11
{'Neighbors': [<Node id=9>], 'Max': -7, 'Temperature': -7, 'Parent': [<Node id=9>]}
PROCESSING
12
{'I': 'Activate', 'Neighbors': [<Node id=9>], 'Temperature': -7, 'Parent': [<Node id=9>], 'Max': -7}
PROCESSING
13
{'Neighbors': [<Node id=10>], 'Max': 17, 'Temperature': 17, 'Parent': [<Node id=10>]}
PROCESSING
14
{'Neighbors': [<Node id=13>], 'Max': 6, 'Temperature': 6, 'Parent': [<Node id=13>]}
PROCESSING
15
{'Neighbors': [<Node id=11>], 'Max': -12, 'Temperature': -12, 'Parent': [<Node id=11>]}
PROCESSING


In [ ]:
net.show()

In [ ]:
print node.memory
#list(node.memory['Neighbors']).remove(node.memory['Parent'])

type(node.memory['Neighbors'])
node.memory['Neighbors'].remove(node.memory['Neighbors'][0])
print node.memory

In [ ]:
print node.memory